In [18]:
!pipenv install langchain_text_splitters langchain_openai langchain_community langchain pypdf chromadb

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langchain_text_splitters...
Resolving langchain_text_splitters...
✔ Installation Succeeded
⠋ Installing langchain_text_splitters...
Installing langchain_openai...
Resolving langchain_openai...
✔ Installation Succeeded
⠋ Installing langchain_openai...
Installing langchain_community...
Resolving langchain_community...
✔ Installation Succeeded
⠋ Installing langchain_community...
Installing langchain...
Resolving langchain...
✔ Installation Succeeded
⠋ Installing langchain...
Installing pypdf...
Resolving pypdf...
✔ Installation Succeeded
⠋ Installing pypdf...
Installing chromadb...
Resolving chromadb...
Added c

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [7]:
import os

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
#Load dos modelos

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200)

In [13]:
#Carregar PDF

pdf_link = "lei2338.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [16]:
#Separar Chunks

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 4000,
  chunk_overlap = 20,
  length_function = len,
  add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [20]:
#Salvar no VectorDB (Chroma)

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")

In [21]:
# Carregar DB

vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

/var/folders/5z/nms2vp9s0_1dm1130xnk264w0000gn/T/ipykernel_37353/351653848.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)


In [22]:
# Load Retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
# Contrução da cadeia de prompt para chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

/var/folders/5z/nms2vp9s0_1dm1130xnk264w0000gn/T/ipykernel_37353/2386744658.py:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [25]:
def ask(question):
  context = retriever.get_relevant_documents(question)
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
  return answer

In [32]:
user_question = input("User: ")
answer = ask(user_question)
print(f"Answer: {answer}")

Answer: A segurança da informação é um dos princípios fundamentais no desenvolvimento, implementação e uso de sistemas de inteligência artificial. Ela é essencial para garantir a confiabilidade, robustez e integridade dos sistemas, bem como para proteger os dados pessoais e sensíveis que podem ser processados por esses sistemas. Medidas adequadas de segurança da informação devem ser adotadas desde a concepção até a operação do sistema, conforme previsto na legislação de proteção de dados e nas diretrizes de governança para sistemas de inteligência artificial.
